In [57]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

In [2]:
data = pd.read_excel("G:\My Drive\Tesis - Mestrado PUCRio\emdat_public_2022_05_25_query_uid-yUy1yp.xlsx")

In [3]:
data.Country

0                            Guatemala
1                            Guatemala
2                            Guatemala
3                               Canada
4                               Canada
                     ...              
5582                       Puerto Rico
5583    United States of America (the)
5584    United States of America (the)
5585    United States of America (the)
5586    United States of America (the)
Name: Country, Length: 5587, dtype: object

In [4]:
data_peru = data[(data["Country"]=="Peru") & (data["Year"]>=2000)]

In [5]:
Disaster_Types = data_peru["Disaster Type"].value_counts()
print(Disaster_Types)

Transport accident        105
Flood                      45
Earthquake                 17
Landslide                  12
Extreme temperature        10
Miscellaneous accident      7
Volcanic activity           4
Epidemic                    4
Drought                     3
Storm                       2
Insect infestation          1
Industrial accident         1
Wildfire                    1
Name: Disaster Type, dtype: int64


In [6]:
extreme_low_temperature = data_peru[data_peru["Disaster Type"]=="Extreme temperature"]

In [195]:
extreme_low_temperature[["Location", "Geo Locations"]].shape

(10, 2)

In [8]:
PUNO_ELT = extreme_low_temperature[extreme_low_temperature["Location"].str.contains('Puno')]

PUNO_ELT["Start Year"].values

array([2003, 2004, 2007, 2010, 2013, 2014, 2015], dtype=int64)

In [48]:
Locations = PUNO_ELT["Geo Locations"].str.replace('(Adm1).', '', regex=False)
Locations = Locations.str.replace(' Apurimac', 'Apurímac', regex=True)
Locations = Locations.str.replace(' Arequipa', 'Arequipa', regex=True)
Locations = Locations.str.replace(' Ayacucho', 'Ayacucho', regex=True)
Locations = Locations.str.replace(' Cusco', 'Cusco', regex=True)
Locations = Locations.str.replace(' Huancavelica', 'Huancavelica', regex=True)
Locations = Locations.str.replace(' Lima', 'Lima', regex=True)
Locations = Locations.str.replace(' Moquegua', 'Moquegua', regex=True)
Locations = Locations.str.replace(' Pasco', 'Pasco', regex=True)
Locations = Locations.str.replace(' Puno', 'Puno', regex=True)
Locations = Locations.str.replace(' Tacna ', 'Tacna', regex=True)
Locations = Locations.str.replace(' Tacna', 'Tacna', regex=True)
Locations = Locations.str.replace(' Ancash', 'Ancash', regex=True)
Locations = Locations.str.replace(' Cajamarca', 'Cajamarca', regex=True)
Locations = Locations.str.replace(' Huanuco', 'Huánuco', regex=True)
Locations = Locations.str.replace(' Junin', 'Junín', regex=True)
Locations = Locations.str.replace(' La Libertad', 'La Libertad', regex=True)
Locations = Locations.str.replace(' Piura', 'Piura', regex=True)
Locations = Locations.str.replace(' San Martin', 'San Martín', regex=True)
Locations = Locations.str.replace(' Callao', 'Callao', regex=True)
Locations = Locations.str.replace(' Ucayali ', 'Ucayali', regex=True)
Locations = Locations.str.replace(' Ucayali', 'Ucayali', regex=True)
Locations = Locations.str.replace(' Lambayeque', 'Lambayeque', regex=True)
Locations = Locations.str.replace(' Loreto', 'Loreto', regex=True)
Locations = Locations.str.replace(' Madre de Dios', 'Madre de Dios', regex=True)


#Locations.iloc[0].split(",")




#PUNO_ELT["Geo Locations"].str.split(pat=",").tolist()

In [67]:
i = -1
Dict = {"Locations2003": [Locations.iloc[0].split(",")]} 

for year in PUNO_ELT["Start Year"]:
    print(year)
    exec(f'Dict["Locations{year}"] = Locations.iloc[i].split(",")')
    i=i+1
    print(i)


2003
0
2004
1
2007
2
2010
3
2013
4
2014
5
2015
6


In [188]:
pop = pd.read_excel("perupop2007.xlsx")

In [189]:
#pop["Dep"] = pop.Dep.astype(str)
pop_var = {"year": [], "total_pop": []}

for year in PUNO_ELT["Start Year"]:
    exec(f'pop_var["total_pop"].append(np.sum(pop.merge(pd.DataFrame(Dict["Locations{year}"], columns=["Dep"]).Dep.astype(str), left_on="Dep", right_on="Dep")["Pop"].values))')
    exec(f'pop_var["year"].append({year})')


In [190]:
pd.DataFrame(pop_var)

,year,total_pop
0,2003,14160029
1,2004,5304887
2,2007,18324487
3,2010,10040551
4,2013,19246005
5,2014,5400447
6,2015,27123303


In [191]:
PUNO_ELT = PUNO_ELT.merge(pd.DataFrame(pop_var), left_on="Start Year", right_on="year")

PUNO_ELT.columns

In [192]:
estimations = PUNO_ELT[["Total Affected", "total_pop", "year"]]
estimations = estimations.rename(columns = {"Total Affected":"total"})

precentage = []
for i in range(7):
    percentage = percentage.append('{:.2%}'.format(estimations.total[i]/estimations.total_pop[i]))

In [193]:
percentage = estimations.total/estimations.total_pop
estimations = pd.concat([estimations, pd.Series(percentage, name="percent")], axis=1)
estimations = pd.concat([estimations, PUNO_ELT["Geo Locations"].str.split(",", expand=True)], axis=1)
estimations

,total,total_pop,year,percent,0,1,2,3,4,5,...,12,13,14,15,16,17,18,19,20,21
0,1839888.0,14160029,2003,0.129935,Apurimac,Arequipa,Ayacucho,Cusco,Huancavelica,Moquegua,...,None,None,None,None,None,None,None,None,None,None
1,2137467.0,5304887,2004,0.402924,Ancash,Apurimac,Arequipa,Ayacucho,Cusco,Huancavelica,...,Puno,Tacna,Ucayali (Adm1).,None,None,None,None,None,None,None
2,884572.0,18324487,2007,0.048273,Apurimac,Arequipa,Ayacucho,Cajamarca,Cusco,Huancavelica,...,Tacna (Adm1).,None,None,None,None,None,None,None,None,None
3,57600.0,10040551,2010,0.005737,Ancash,Apurimac,Arequipa,Ayacucho,Cusco,Huancavelica,...,Pasco,Puno,Tacna,Ucayali (Adm1).,None,None,None,None,None,None
4,92701.0,19246005,2013,0.004817,Apurimac,Ayacucho,Cusco,Huancavelica,Junin,Moquegua,...,None,None,None,None,None,None,None,None,None,None
5,109257.0,5400447,2014,0.020231,Amazonas,Ancash,Apurimac,Arequipa,Ayacucho,Cajamarca,...,Lambayeque,Lima,Loreto,Moquegua,Pasco,Piura,Puno,San Martin,Tacna,Ucayali (Adm1).
6,200620.0,27123303,2015,0.007397,Apurimac,Arequipa,Ayacucho,Cusco,Huancavelica,Lima,...,None,None,None,None,None,None,None,None,None,None


In [194]:
estimations.to_excel("data_large_scale.xlsx")

In [ ]:
data_peru[data_peru["Disaster Type"]=="Extreme temperature"].to_excel("Extreme temperature.xlsx")

In [ ]:
sns.set_theme(style="darkgrid", palette="Set3")

In [ ]:
sns.barplot(y=[Disaster_Types.index[i] for i in (1, 2, 3, 9)]  ,x=[Disaster_Types.values[i] for i in (1, 2, 3, 9)] )

plt.ylabel('Disaster Type')
plt.xlabel('Suddenly onset clime-related disasters 2000-2022')
plt.title('Cummulative Disaster Frequency Peru',color = 'blue',fontsize=15)
plt.savefig('Cummulative Disaster Frequency Peru.png', dpi = 1200, bbox_inches="tight")

In [ ]:
Disaster_per_Year = data_peru["Year"].value_counts()

sns.pointplot(x=Disaster_per_Year.index, y=Disaster_per_Year.values)
plt.ylabel('Suddenly onset clime-related disasters')
plt.xlabel('Year (2000-2022)')
plt.xticks(rotation=45)
plt.title('Disaster Frequency per Year Peru',color = 'blue',fontsize=15)
plt.savefig('Disaster Frequency per Year Peru.png', dpi = 1200, bbox_inches="tight")
plt.tight_layout()

In [ ]:
Disaster_per_Year

In [ ]:

# Python program to convert .tsv file to .csv file
# importing re library
import re
  
# reading given tsv file
with open("G:\My Drive\Tesis - Mestrado PUCRio\Databases\earthquakes-2022-05-29_23-58-03_-0300.tsv", 'r') as myfile:  
  with open("G:\My Drive\Tesis - Mestrado PUCRio\Databases\earthquakes-2022-05-29_23-58-03_-0300.csv", 'w') as csv_file:
    for line in myfile:
        
      # Replace every tab with comma
      fileContent = re.sub("\t", ",", line)
        
      # Writing into csv file
      csv_file.write(fileContent)
  
# output
print("Successfully made csv file")